In [1]:
import os , sys
sys.path.append(os.path.join(os.getcwd(),'..'))

In [5]:
import pandas as pd
from scripts.sentiment_labeler import assign_sentiment_label

from textblob import TextBlob


In [3]:
data = pd.read_csv('C:/kifiya ai/stock-market-predictions/src/data/raw_analyst_ratings.csv')

In [14]:
apple_data = pd.read_csv('../src/data/yfinance_data/AAPL_historical_data.csv')
amazon_data = pd.read_csv('../src/data/yfinance_data/AMZN_historical_data.csv')
google_data = pd.read_csv('../src/data/yfinance_data/GOOG_historical_data.csv')
meta_data = pd.read_csv('../src/data/yfinance_data/META_historical_data.csv')
microsoft_data = pd.read_csv('../src/data/yfinance_data/MSFT_historical_data.csv')
nvidia_data = pd.read_csv('../src/data/yfinance_data/NVDA_historical_data.csv')
tesla_data = pd.read_csv('../src/data/yfinance_data/TSLA_historical_data.csv')

In [6]:
data['date'] = pd.to_datetime(data['date'],  errors='coerce')
data['sentiment_label'] = data['headline'].apply(lambda headline: assign_sentiment_label(TextBlob(headline).sentiment.polarity))
data.head()


,Unnamed: 0,headline,url,publisher,date,stock,sentiment_label
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A,Neutral
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A,Neutral
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A,Neutral
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A,Neutral
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A,Neutral


In [19]:
import pandas as pd

# Convert `data['date']` to datetime and remove timezone information
data['date'] = pd.to_datetime(data['date'], utc=True).dt.tz_localize(None)

# Ensure stock data 'Date' columns are in datetime format
stock_datasets = [tesla_data, google_data, amazon_data, microsoft_data, nvidia_data, meta_data, apple_data]
for stock_data in stock_datasets:
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])

# Calculate Daily Returns for each company
for stock_data, company_name in zip(stock_datasets, ['tesla', 'google', 'amazon', 'microsoft', 'nvidia', 'meta', 'apple']):
    stock_data[f'{company_name}_daily_return'] = stock_data['Adj Close'].pct_change()

# Function to merge news data with stock data and calculate correlation
def analyze_sentiment_and_stock_returns(stock_data, news_data, company_name):
    # Filter news data to only relevant dates in stock data
    min_date, max_date = stock_data['Date'].min(), stock_data['Date'].max()
    filtered_news_data = news_data[(news_data['date'] >= min_date) & (news_data['date'] <= max_date)]
    
    # Merge the news data with stock data
    merged_data = pd.merge(
        filtered_news_data[['date', 'sentiment_label']],
        stock_data[['Date', f'{company_name}_daily_return']],
        left_on='date',
        right_on='Date',
        how='inner'
    )
    
    # Map sentiment labels to scores
    sentiment_mapping = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
    merged_data['sentiment_score'] = merged_data['sentiment_label'].map(sentiment_mapping)
    
    # Calculate correlation
    correlation = merged_data['sentiment_score'].corr(merged_data[f'{company_name}_daily_return'])
    return correlation, merged_data

# Perform analysis for each company
companies_data = {
    'tesla': tesla_data,
    'google': google_data,
    'amazon': amazon_data,
    'microsoft': microsoft_data,
    'nvidia': nvidia_data,
    'meta': meta_data,
    'apple': apple_data,
}

correlation_results = {}
merged_data_results = {}

for company_name, stock_data in companies_data.items():
    correlation, merged_data = analyze_sentiment_and_stock_returns(stock_data, data, company_name)
    correlation_results[company_name.capitalize()] = correlation
    merged_data_results[company_name] = merged_data

# Print the correlations
for company, correlation in correlation_results.items():
    print(f"Correlation between sentiment and {company} stock returns: {correlation}")

# Store the results in a DataFrame
correlation_df = pd.DataFrame(list(correlation_results.items()), columns=['Company', 'Correlation'])
print(correlation_df)


c:\kifiya ai\stock-market-predictions\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:2914: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\kifiya ai\stock-market-predictions\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:2773: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\kifiya ai\stock-market-predictions\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:2773: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
c:\kifiya ai\stock-market-predictions\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:2914: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\kifiya ai\stock-market-predictions\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:2773: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\kifiya ai\stock-market-predictions\.venv\Lib\site-packages\numpy\

Correlation between sentiment and Tesla stock returns: nan
Correlation between sentiment and Google stock returns: nan
Correlation between sentiment and Amazon stock returns: nan
Correlation between sentiment and Microsoft stock returns: nan
Correlation between sentiment and Nvidia stock returns: nan
Correlation between sentiment and Meta stock returns: nan
Correlation between sentiment and Apple stock returns: nan
     Company  Correlation
0      Tesla          NaN
1     Google          NaN
2     Amazon          NaN
3  Microsoft          NaN
4     Nvidia          NaN
5       Meta          NaN
6      Apple          NaN


c:\kifiya ai\stock-market-predictions\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:2914: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\kifiya ai\stock-market-predictions\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:2773: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\kifiya ai\stock-market-predictions\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:2773: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
c:\kifiya ai\stock-market-predictions\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:2914: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\kifiya ai\stock-market-predictions\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:2773: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\kifiya ai\stock-market-predictions\.venv\Lib\site-packages\numpy\